In [21]:
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:

# Function to parse the log file
def parse_log_file(log_file_path):
    iterations = []
    mious = []
    maccs = []
    learning_rates = []

    last_iter = None
    last_lr = None

    with open(log_file_path, 'r') as f:
        for line in f:
            # Parse training iteration and learning rate
            train_match = re.search(r"Iter\(train\)\s*\[\s*(\d+)/\d+\]\s*lr:\s*([0-9\.e+-]+)", line)
            if train_match:
                last_iter = int(train_match.group(1))
                last_lr = float(train_match.group(2))
                continue

            # Parse validation metrics (mIoU and mAcc)
            val_match = re.search(r"mIoU:\s*(\d+\.\d+)\s*mAcc:\s*(\d+\.\d+)", line)
            if val_match:
                if last_iter is not None:
                    iterations.append(last_iter)
                    mious.append(float(val_match.group(1)))
                    maccs.append(float(val_match.group(2)))
                    learning_rates.append(last_lr)

    return iterations, mious, maccs, learning_rates

# New function to plot interactively with Plotly
def plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, x_range):
    # Create a figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Top-1 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mAcc, name='Mean Accuracy', line=dict(color='blue')),
        secondary_y=False,
    )

    # Add Top-5 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mIoU, name='Mean IoU', line=dict(color='orange')),
        secondary_y=False,
    )

    # Add Learning Rate trace on secondary y-axis
    fig.add_trace(
        go.Scatter(x=epochs, y=lrs, name='Learning Rate', line=dict(color='green')),
        secondary_y=True,
    )

    # Update layout for interactivity and axes
    fig.update_layout(
        title='Validation Performance and Training Learning Rate vs Epoch',
        xaxis_title='Epoch',
        yaxis_title='Performance (%)',
        yaxis2_title='Learning Rate',
        hovermode='x unified',  # Vertical crosshair on x-hover, shows values for all lines
        legend=dict(yanchor='top', y=1.15, xanchor='right', x=1.15),  # Top-right legend
    )

    # Set x-axis ranges
    fig.update_xaxes(range=x_range)
    
    # Set y-axis ranges
    fig.update_yaxes(range=[0, 100], dtick=10, secondary_y=False)
    fig.update_yaxes(secondary_y=True)  # Auto-range for LR

    # Show the figure (interactive in Jupyter)
    fig.show()


In [23]:
adam_range = [0, 310]
sgd_range = [0, 120000]

In [24]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, Error: Wrong Training Loop, so the learning rate stayed constant throughout
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes/20250814_203128/20250814_203128.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [25]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8, Error: Wrong Training Loop, so the learning rate stayed constant throughout
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes/20250814_203149/20250814_203149.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [26]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, 120K - 65.08 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce/20250819_174704/20250819_174704.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [27]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6 - 65.08 Best mIoU. Continued for extra 40k iters
# Finetuning for an extra 40k iterations may not work. This starts the learning rate schedule over, so 
# minima exploration jumps are larger. Going to keep training just for observation though.
# 65.35 Best mIoU 
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce_scratch/20250821_114549/20250821_114549.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,40000])

In [28]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, 160K - 66.74 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce/20250821_121920/20250821_121920.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [29]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8 - 65.85 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce/20250819_175241/20250819_175241.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [30]:
# SEBNet BaselineHead SGD, Scratch, Batch Size 8 - 65.54 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce_scratch/20250819_174850/20250819_174850.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [31]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8, OHEM Cross Entropy - 67.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ohem/20250819_174814/20250819_174814.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [32]:
# SEBNet PIDNet SGD, Pretrained, Batch Size 6
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250821_094855/20250821_094855.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [33]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, CE Cross Entropy - 72.15 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce/20250822_155456/20250822_155456.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [34]:
# SEBNet BaselineHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 72.26 mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce_scratch/20250822_155431/20250822_155431.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [35]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8, CE Cross Entropy - 71.01 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce/20250822_155554/20250822_155554.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [36]:
# SEBNet BaselineHead SGD, Scratch, Batch Size 8, CE Cross Entropy - 70.99 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce_scratch/20250822_155524/20250822_155524.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [37]:
# SEBNet BaselineCASENetHead SGD, Scratch, Batch Size 6, CE Cross Entropy
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-casenet-head_1xb6_cityscapes/20250823_235330/20250823_235330.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [38]:
# SEBNet BaselineDFFHead SGD, Scratch, Batch Size 6, CE Cross Entropy
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-dff-head_1xb6_cityscapes/20250823_235446/20250823_235446.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [39]:
# SEBNet BaselineBEMHead SGD, Scratch, Batch Size 6, CE Cross Entropy
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-bem-head_1xb6_cityscapes/20250823_235417/20250823_235417.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [40]:
# SEBNet BaselineMIMIRHead SGD, Scratch, Batch Size 6, CE Cross Entropy
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-mimir-head_1xb6_cityscapes/20250823_235809/20250823_235809.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])